# Script Machine Debugging Tool (Libbitcoin c++)

<hr style="border: 0.5px dashed #000;">

In [ ]:
// Compiler & linker information for c++ interpreter.
#pragma cling add_include_path("/usr/local/include")
#pragma cling add_library_path("/usr/local/lib")
#pragma cling load("bitcoin","secp256k1","pthread","boost_chrono-mt","boost_date_time-mt","boost_filesystem","boost_iostreams-mt","boost_locale-mt","boost_log-mt","boost_program_options-mt","boost_regex-mt","boost_system","boost_thread-mt")

// Libbitcoin-System.
#include <bitcoin/bitcoin.hpp> 

// Libbitcoin-Script Machine Script.
#include "supporting_scripts/script_machine.cpp"

### 1. Previous transaction output (UTXO).

**Previous output script** (fetch by `txid` & `index`)

In [ ]:
!bx fetch-tx 8231a9027eca6f2bd7bdf712cd2368f0b6e8dd6005b6b348078938042178ffed -f json | jq -r  ".transaction.outputs[0].script" | bx script-encode


In [ ]:
auto previous_output_script_data = bc::to_chunk(
  bc::base16_literal("a914dcdc2f89b96c420751e3750da7d5073a81b1694687"));
bc::chain::script previous_output_script;
previous_output_script.from_data(previous_output_script_data, false);


**Previous output amount** (fetch by `txid` & `index`)

In [ ]:
!bx fetch-tx 8231a9027eca6f2bd7bdf712cd2368f0b6e8dd6005b6b348078938042178ffed -f json | jq -r  ".transaction.outputs[0].value"


In [ ]:
uint64_t previous_output_amount = 129800000;


### 2. Spending transaction.

In [ ]:
// Paste your raw transaction data.
auto spending_transaction_data = bc::to_chunk(
  bc::base16_literal("01000000000101edff78210438890748b3b60560dde8b6f06823cd12f7bdd72b6fca7e02a9318200000000171600149a19a31c2fda7d0c30215ec954a20a542aa84ad3ffffffff016003b807000000001976a914bbef244bcad13cffb68b5cef3017c7423675552288ac0247304402207ecbb796a2bc706d90e2ed7efb58f59822bdc4c253b91f6eecd26ca5df1a6bb60220700b737f3c49b2f21bb228fadeab786e2ac78fd87890ede3f5d299e81880d9630121026ccfb8061f235cc110697c0bfb3afb99d82c886672f6b9b5393b25a434c0cbf300000000"));
bc::chain::transaction spending_transaction;
auto tx_set = spending_transaction.from_data(spending_transaction_data, true, true); // wire=true, witness=true


**Index of spending input.**

In [ ]:
uint32_t input_index = 0;

### 3. Evaluate input, output, p2sh and witness script runs.

Run following cell to evaluate the `previous output` | `input` script(s).

In [ ]:
bc::code ec = evaluate_in_out_p2sh_witness_runs(previous_output_script, previous_output_amount,
  spending_transaction, input_index, bc::machine::rule_fork::all_rules);
std::cout << ec.message() << std::endl;
